# Data Reading
Reading the given datasets into pandas dataframes


In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

train_data = pd.read_csv('../input/yds_train2018.csv')
test_data = pd.read_csv('../input/yds_test2018.csv')
promotional_data = pd.read_csv('../input/promotional_expense.csv')
holidays_data = pd.read_excel('../input/holidays.xlsx')


# Resampling Train Data Set
Aggregating weekly sales into months and sorting them on Countries ascending order

In [ ]:
data_monthly = train_data.groupby([ 'Year', 'Month', 'Product_ID', 'Country'], as_index=False)['Sales'].sum().sort_values(['Country','Year','Month','Product_ID'], ascending=True)
data_monthly.head(10)

# Creating Master Dataset 

The objective is to add the information from holidays dataset and promotional expense dataset as features into train and test datasets

Initially the Date column of holidays dataset is converted to datetime object to derive month and year features from this datetime object

The holiday count per month is counted and added as a seperate feature in the holiday dataset


In [ ]:
holidays_data['Date'] = pd.to_datetime(holidays_data['Date'])
holidays_data['Year'] = pd.DatetimeIndex(holidays_data['Date']).year
holidays_data['Month'] = pd.DatetimeIndex(holidays_data['Date']).month

holidays_data = holidays_data.groupby([ 'Year', 'Month', 'Country'], as_index=False)['Holiday'].count()
holidays_data.head(10)

Performing** Left Join **on **aggregated_monthly_train** and **test dataset** with **promotional dataset** on Year Month ProductID and Country

In [ ]:
temp1_data = pd.merge(data_monthly, promotional_data, how='left', on=['Year','Month','Product_ID','Country'])
temp2_data = pd.merge(test_data, promotional_data, how='left', on=['Year','Month','Product_ID','Country'])

print('Edited Train Dataset with additional Expense_Price Feature')
print(temp1_data.head(10),'\n')
print('Edited Test Dataset with additional Expense_Price Feature')
print(temp2_data.head(10))

Merging the above train and test datasets with holidays dataset to form final master datasets

In [ ]:
final1_data = pd.merge(temp1_data, holidays_data, how='left', left_on=['Year','Month','Country'], right_on = ['Year','Month','Country'])
final2_data = pd.merge(temp2_data, holidays_data, how='left', on=['Year','Month','Country'])

print('Master Train Dataset with additional Expense_Price Feature, Holiday Feature')
print(final1_data.head(10),'\n')
print('Master Test Dataset with additional Expense_Price Feature, Holiday Feature')
print(final2_data.head(10))

# Data Cleaning
For the Holiday Feature, the NaN values are replaced by 0, since imputing them will not make sense as those months will not have that many holidays

For the Expense_Price Feature, mean and median imputation is compared and median imputation is chosen since majority of values are closer to computed mean than the computed median

In [ ]:
#Data Imputation Training Data
#Holiday Replacing NaN with 0 since no holidays in that month denote 0 Holiday
final1_data['Holiday'].fillna(0,inplace=True)
#Expense Price using Median as Imputation Parameter
mean = final1_data['Expense_Price'].mean()
median = final1_data['Expense_Price'].median()
final1_data['Expense_Price'].fillna(median,inplace=True)

#Data Imputation Test Data
#Holiday Replacing NaN with 0 since no holidays in that month denote 0 Holiday
final2_data['Holiday'].fillna(0,inplace=True)
#Expense Price using Median as Imputation Parameter
mean = final2_data['Expense_Price'].mean()
median = final2_data['Expense_Price'].median()
final2_data['Expense_Price'].fillna(median,inplace=True)

print('Cleaned Master Train Dataset')
print(final1_data.head(10),'\n')
print('Cleaned Test Dataset')
print(final2_data.head(10))

Writing final test and train master datasets to csv for further analysis with RapidMiner Studio

In [ ]:
final1_data.to_csv('train_data_addedFeatures_mean.csv', sep=',', encoding='utf-8',header=True, index_label='id')
final2_data.to_csv('test_with_addedFeatures_mean.csv', sep=',', encoding='utf-8',header=True, index_label='id')

# Data Exploration and Data Visualization